In [21]:
import os

os.environ["CFG_FILE"] = os.path.expanduser(
    "~/workspace/workspace/projects/ridge/config.dev.yml"
)
os.environ["DEPLOY_ENV"] = "dev"

from supplier.models.db import Supplier, engine, get_session

In [14]:
from sqlalchemy.schema import CreateTable, CreateIndex
from sqlalchemy import select
from sqlalchemy import func

In [6]:
print(CreateTable(Supplier.__table__).compile(engine))


CREATE TABLE supplier (
	id SERIAL NOT NULL, 
	tenant_id INTEGER NOT NULL, 
	created_at TIMESTAMP WITHOUT TIME ZONE NOT NULL, 
	updated_at TIMESTAMP WITHOUT TIME ZONE NOT NULL, 
	name VARCHAR NOT NULL, 
	type suppliertype NOT NULL, 
	class_ supplierclass NOT NULL, 
	priority supplierpriority NOT NULL, 
	address_country VARCHAR, 
	address_administrative_area_level_1 VARCHAR, 
	address_administrative_area_level_2 VARCHAR, 
	address_administrative_area_level_3 VARCHAR, 
	address_street_address VARCHAR, 
	address_postal_code VARCHAR, 
	contact_first_name VARCHAR, 
	contact_last_name VARCHAR, 
	contact_middle_name VARCHAR, 
	contact_full_name VARCHAR, 
	contact_email VARCHAR, 
	contact_phone_number VARCHAR, 
	website VARCHAR, 
	note VARCHAR NOT NULL, 
	deactivate BOOLEAN NOT NULL, 
	vat_type vattype NOT NULL, 
	vat_value FLOAT, 
	vat_is_required BOOLEAN NOT NULL, 
	tax_name VARCHAR, 
	tax_address VARCHAR, 
	tax_identifier VARCHAR, 
	PRIMARY KEY (id), 
	FOREIGN KEY(tenant_id) REFERENCES ten

In [7]:
for index in Supplier.__table__.indexes:
    print(CreateIndex(index).compile(engine))

CREATE INDEX idx_supplier_name_gin ON supplier USING gin (to_tsvector('english', name))
CREATE INDEX ix_supplier_updated_at ON supplier (updated_at)
CREATE INDEX ix_supplier_created_at ON supplier (created_at)
CREATE INDEX ix_supplier_type ON supplier (type)


In [16]:
print(Supplier.name.bool_op("@@"))

<function Operators.op.<locals>.against at 0x1152a65c0>


In [32]:
q = select(Supplier).where(Supplier.name.bool_op("@@")(func.plainto_tsquery("ninh")))

In [39]:
from sqlalchemy import text

# compile the select with literal bound parameters so we get concrete SQL
compiled = q.compile(engine, compile_kwargs={"literal_binds": True})
explain_sql = "EXPLAIN (ANALYZE, BUFFERS, VERBOSE, FORMAT TEXT) " + str(compiled)

print(explain_sql)

# execute EXPLAIN and print the plan lines
with get_session() as session:
    explain_rows = session.execute(text(explain_sql)).all()

for row in explain_rows:
    print(row[0])
print("=" * 80)

explain_sql = explain_sql.replace(
    "WHERE supplier.name @@", "WHERE to_tsvector('english', supplier.name) @@"
)
# execute EXPLAIN and print the plan lines
with get_session() as session:
    explain_rows = session.execute(text(explain_sql)).all()

for row in explain_rows:
    print(row[0])

EXPLAIN (ANALYZE, BUFFERS, VERBOSE, FORMAT TEXT) SELECT supplier.id, supplier.tenant_id, supplier.created_at, supplier.updated_at, supplier.name, supplier.type, supplier.class_, supplier.priority, supplier.address_country, supplier.address_administrative_area_level_1, supplier.address_administrative_area_level_2, supplier.address_administrative_area_level_3, supplier.address_street_address, supplier.address_postal_code, supplier.contact_first_name, supplier.contact_last_name, supplier.contact_middle_name, supplier.contact_full_name, supplier.contact_email, supplier.contact_phone_number, supplier.website, supplier.note, supplier.deactivate, supplier.vat_type, supplier.vat_value, supplier.vat_is_required, supplier.tax_name, supplier.tax_address, supplier.tax_identifier 
FROM supplier 
WHERE supplier.name @@ plainto_tsquery('ninh')
2025-09-17 20:52:30,874 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-17 20:52:30,875 INFO sqlalchemy.engine.Engine EXPLAIN (ANALYZE, BUFFERS, VERBOSE

In [43]:
q = select(Supplier).where(
    func.to_tsvector("english", Supplier.name).bool_op("@@")(
        func.plainto_tsquery("ninh bay")
    )
)

with get_session() as session:
    rs = session.execute(q).scalars().all()
    for r in rs:
        print(r.id, r.name)

2025-09-18 06:27:40,679 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-18 06:27:40,680 INFO sqlalchemy.engine.Engine SELECT supplier.id, supplier.tenant_id, supplier.created_at, supplier.updated_at, supplier.name, supplier.type, supplier.class_, supplier.priority, supplier.address_country, supplier.address_administrative_area_level_1, supplier.address_administrative_area_level_2, supplier.address_administrative_area_level_3, supplier.address_street_address, supplier.address_postal_code, supplier.contact_first_name, supplier.contact_last_name, supplier.contact_middle_name, supplier.contact_full_name, supplier.contact_email, supplier.contact_phone_number, supplier.website, supplier.note, supplier.deactivate, supplier.vat_type, supplier.vat_value, supplier.vat_is_required, supplier.tax_name, supplier.tax_address, supplier.tax_identifier 
FROM supplier 
WHERE to_tsvector(%(to_tsvector_1)s::REGCONFIG, supplier.name) @@ plainto_tsquery(%(plainto_tsquery_1)s::VARCHAR)
2025-09-18 06:27

In [48]:
with get_session() as session:
    print(
        session.execute(
            select(Supplier.id, func.to_tsquery("english", "Hello world")).limit(1)
        ).all()
    )

2025-09-18 06:48:50,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-18 06:48:50,165 INFO sqlalchemy.engine.Engine SELECT supplier.id, to_tsquery(%(to_tsquery_2)s::REGCONFIG, %(to_tsquery_3)s::VARCHAR) AS to_tsquery_1 
FROM supplier 
 LIMIT %(param_1)s::INTEGER
2025-09-18 06:48:50,165 INFO sqlalchemy.engine.Engine [generated in 0.00046s] {'to_tsquery_2': 'english', 'to_tsquery_3': 'Hello world', 'param_1': 1}
2025-09-18 06:48:50,170 INFO sqlalchemy.engine.Engine ROLLBACK


ProgrammingError: (psycopg.errors.SyntaxError) syntax error in tsquery: "Hello world"
[SQL: SELECT supplier.id, to_tsquery(%(to_tsquery_2)s::REGCONFIG, %(to_tsquery_3)s::VARCHAR) AS to_tsquery_1 
FROM supplier 
 LIMIT %(param_1)s::INTEGER]
[parameters: {'to_tsquery_2': 'english', 'to_tsquery_3': 'Hello world', 'param_1': 1}]
(Background on this error at: https://sqlalche.me/e/20/f405)